In [8]:
import httpx
import polars as pl
import pycountry
from sklearn.model_selection import train_test_split

from civil_unrest_correlation_analysis.utils.building import build_dataset
from civil_unrest_correlation_analysis.utils.model import build_feature_df, import_pipeline

oecd_csv = ('data/final/oecd.csv')
acled_csv = ('data/final/acled.csv')
data_csv = ('data/final/data.csv')
model_pkl = ('random_forest.pkl')
data = build_dataset(oecd_csv, acled_csv, data_csv, read_data_csv=False)
X = data.drop(['iso', 'year_month', 'incidents'])
y = data['incidents']
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipe = import_pipeline(X_train, y_train, model_pkl)
coeffs = build_feature_df(pipe)
print(f'R2 Score: {pipe.score(X_test, y_test)}')

R2 Score: 0.9207561456095029


In [62]:
import sklearn

In [12]:
import polars as pl
from civil_unrest_correlation_analysis.utils.building import build_filtered_acled_events
acled = pl.read_csv('data/final/acled.csv', schema_overrides={'iso': pl.String})
df = build_filtered_acled_events(acled, '203', '2019-01', '2021-01')

In [18]:
acled.filter(pl.col('iso') == '203').select(pl.col('admin2')).unique()

admin2
str
"""Strakonice"""
"""Jablonec nad Nisou"""
"""Brno-venkov"""
"""Chomutov"""
"""Brno-mesto"""
…
"""Ceske Budejovice"""
"""Melnik"""
"""Kladno"""


In [1]:
from contextlib import asynccontextmanager
from typing import Any

import polars as pl
from fastapi import FastAPI, HTTPException, Query
from fastapi.middleware.cors import CORSMiddleware

from civil_unrest_correlation_analysis.utils.building import (
    build_acled_events_dict,
    build_countries_dict,
    build_dataset,
    clean_acled,
    clean_oecd,
    raw_acled,
    build_geojson_dict
)
from civil_unrest_correlation_analysis.schema import (
    AcledEvent,
    BaseModel,
    CountryMeta,
    OecdMetric,
    SnapshotResponse,
)

OECD_CSV = 'data/final/oecd.csv'
ACLED_CSV = 'data/final/acled.csv'
DATA_CSV = 'data/final/data.csv'
DATAFRAMES: dict[str, pl.DataFrame] = {}
COUNTRIES: dict[str, CountryMeta] = {}
COUNTRIES_GEO: dict[str, Any] = {}
LIFESPAN_OBJS: list[dict[str,Any]] = []
ORIGINS = ['http://localhost:5173']

DATAFRAMES['raw_acled'] = raw_acled(ACLED_CSV)
DATAFRAMES['clean_acled'] = clean_acled(ACLED_CSV)
DATAFRAMES['oecd'] = clean_oecd(OECD_CSV)
DATAFRAMES['data'] = build_dataset(acled_csv=ACLED_CSV,
                                oecd_csv=DATA_CSV,
                                data_csv=DATA_CSV)
LIFESPAN_OBJS.append(DATAFRAMES)
if DATAFRAMES.get('raw_acled') is not None:
    COUNTRIES.update(build_countries_dict(DATAFRAMES['raw_acled']))
    COUNTRIES_GEO.update(build_geojson_dict(COUNTRIES))
    LIFESPAN_OBJS.append(COUNTRIES)


In [ ]:
from civil_unrest_correlation_analysis.utils.building import 

In [ ]:
from civil_unrest_correlation_analysis.utils.building import build_snapshot
acled = DATAFRAMES['raw_acled']
iso = '203'
start = '2020-01'
end = '2021-01'
snapshot = build_snapshot(COUNTRIES_GEO, acled_slice, '840', '2020-01', '2021-01', 'ADM1')

iso: 840
geojson feature count: 56
first shapeName: Mississippi
type(first shapeName): <class 'str'>
properties keys: ['shapeName', 'shapeISO', 'shapeID', 'shapeGroup', 'shapeType']
cleaned_acled_df cols: ['iso', 'year_month', 'event_date', 'admin1', 'location', 'event_type', 'sub_event_type', 'fatalities', 'notes', 'cleaned_name']
cleaned_region_df cols: ['shapeName', 'cleaned_name']
ACLED cleaned_name sample:
shape: (0, 1)
┌──────────────┐
│ cleaned_name │
│ ---          │
│ str          │
╞══════════════╡
└──────────────┘
REGION cleaned_name sample:
shape: (10, 2)
┌──────────────┬──────────────┐
│ cleaned_name ┆ shapeName    │
│ ---          ┆ ---          │
│ str          ┆ str          │
╞══════════════╪══════════════╡
│ pennsylvania ┆ Pennsylvania │
│ alabama      ┆ Alabama      │
│ texas        ┆ Texas        │
│ maryland     ┆ Maryland     │
│ iowa         ┆ Iowa         │
│ wyoming      ┆ Wyoming      │
│ georgia      ┆ Georgia      │
│ montana      ┆ Montana      │
│ tennesse

In [4]:
map = snapshot.map_spec['map']
type(map)

geoacled.chart.Choropleth

In [ ]:
map.

,geometry,shapeName,shapeISO,shapeID,shapeGroup,shapeType
0,"POLYGON ((14.30231 50.12981, 14.30232 50.12984...",Hlavní město Praha,CZ-10,73172107B16313596965023,CZE,ADM1
1,"POLYGON ((15.4971 49.86111, 15.49445 49.86009,...",Středočeský kraj,CZ-20,73172107B58641114734330,CZE,ADM1
2,"POLYGON ((13.76572 49.51394, 13.76911 49.51113...",Jihočeský kraj,CZ-31,73172107B46425968394697,CZE,ADM1
3,"POLYGON ((13.40676 50.08958, 13.40904 50.08862...",Plzeňský kraj,CZ-32,73172107B93878397098511,CZE,ADM1
4,"POLYGON ((13.30175 50.09935, 13.30148 50.09733...",Karlovarský kraj,CZ-41,73172107B80386947470046,CZE,ADM1
5,"POLYGON ((14.48643 50.50499, 14.46927 50.50935...",Ústecký kraj,CZ-42,73172107B34062378650272,CZE,ADM1
6,"POLYGON ((15.14693 50.52268, 15.14114 50.52265...",Liberecký kraj,CZ-51,73172107B98776772787889,CZE,ADM1
7,"POLYGON ((15.43738 50.10989, 15.43824 50.11205...",Královéhradecký kraj,CZ-52,73172107B56271871797203,CZE,ADM1
8,"POLYGON ((15.4971 49.86111, 15.50106 49.86242,...",Pardubický kraj,CZ-53,73172107B96941531518499,CZE,ADM1
9,"POLYGON ((14.93201 49.54938, 14.93091 49.55419...",Kraj Vysočina,CZ-63,73172107B33107872828953,CZE,ADM1


In [20]:
import httpx
url = 'http://127.0.0.1:8000/snapshot'
params = {
    'iso': '203',
    'start': '2019-01',
    'end': '2021-01'
}
r = httpx.get(url, params=params)
print(r.json())

{'iso': '203', 'country': 'Czech Republic', 'start': '2019-01', 'end': '2021-01', 'acled_events': [{'event_date': '2020-09-27', 'admin1': 'Prague', 'location': 'Prague', 'event_type': 'Protests', 'sub_event_type': 'Peaceful protest', 'fatalities': 0, 'notes': 'On 27 September 2020, several hundred activists gathered for a rally in Prague, protesting against coronavirus-related measures and restrictions imposed by the government.'}, {'event_date': '2020-11-17', 'admin1': 'Prague', 'location': 'Prague', 'event_type': 'Protests', 'sub_event_type': 'Peaceful protest', 'fatalities': 0, 'notes': 'On 17 November 2020, thousands of people protested in Prague against the coronavirus measures and against the government.'}, {'event_date': '2020-04-29', 'admin1': 'Olomouc', 'location': 'Sumperk', 'event_type': 'Protests', 'sub_event_type': 'Peaceful protest', 'fatalities': 0, 'notes': "On 29 April 2020, activists gathered in Sumperk, protesting against allegedly chaotic measures taken by the gover

In [9]:
url = 'http://127.0.0.1:8000/countries'
r = httpx.get(url)
print(r.json())

[{'iso': '032', 'name': 'Argentina'}, {'iso': '036', 'name': 'Australia'}, {'iso': '040', 'name': 'Austria'}, {'iso': '056', 'name': 'Belgium'}, {'iso': '076', 'name': 'Brazil'}, {'iso': '124', 'name': 'Canada'}, {'iso': '152', 'name': 'Chile'}, {'iso': '156', 'name': 'China'}, {'iso': '170', 'name': 'Colombia'}, {'iso': '188', 'name': 'Costa Rica'}, {'iso': '191', 'name': 'Croatia'}, {'iso': '203', 'name': 'Czech Republic'}, {'iso': '208', 'name': 'Denmark'}, {'iso': '233', 'name': 'Estonia'}, {'iso': '246', 'name': 'Finland'}, {'iso': '250', 'name': 'France'}, {'iso': '276', 'name': 'Germany'}, {'iso': '300', 'name': 'Greece'}, {'iso': '348', 'name': 'Hungary'}, {'iso': '352', 'name': 'Iceland'}, {'iso': '356', 'name': 'India'}, {'iso': '360', 'name': 'Indonesia'}, {'iso': '372', 'name': 'Ireland'}, {'iso': '376', 'name': 'Israel'}, {'iso': '380', 'name': 'Italy'}, {'iso': '392', 'name': 'Japan'}, {'iso': '428', 'name': 'Latvia'}, {'iso': '440', 'name': 'Lithuania'}, {'iso': '442', '

In [5]:
if isinstance(coeffs, pl.DataFrame):
    coeffs.sort(by='importance',
                descending=True).head().plot.bar(x='feature',
                                                 y='importance').show()

alt.Chart(...)

In [12]:
import altair as alt

In [16]:
alt.Chart(coeffs).mark_bar(color='firebrick').encode(
    alt.X('importance', type='quantitative'),
    alt.Y('feature', type='nominal').sort('-x')
    
)

alt.Chart(...)

### Global Acute Food Insecurity
* https://data.humdata.org/dataset/global-acute-food-insecurity-country-data
* Only covers developing nations, not compatible with current OECD countries

In [6]:
#ipc = pl.read_csv('data/ipc_global_national_long_latest_food_insecurity.csv')

### Global Mobility Report (Google):
* https://data.humdata.org/dataset/google-mobility-report
* Only covers 2020-2022

In [7]:
#global_mobility_report = pl.read_csv('data/inspected/Global_Mobility_Report.csv')

In [42]:
acled

event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,actor2,assoc_actor_2,inter2,interaction,civilian_targeting,iso,region,country,admin1,admin2,admin3,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp
str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,i64,str,str,str,i64,str,i64
"""SAF12093""","""2020-01-16""",2020,1,"""Demonstrations""","""Riots""","""Violent demonstration""","""Rioters (South Africa)""","""""","""Rioters""","""""","""""","""""","""Rioters only""","""""","""710""","""Southern Africa""","""South Africa""","""Western Cape""","""Central Karoo""","""Beaufort West""","""Beaufort West""",-32.3536,22.5806,1,"""News24 (South Africa); Citizen…","""National""","""On 16 January 2020, parents de…",0,"""crowd size=no report""",1579553213
"""SAF12089""","""2020-01-13""",2020,2,"""Demonstrations""","""Protests""","""Peaceful protest""","""Protesters (South Africa)""","""""","""Protesters""","""""","""""","""""","""Protesters only""","""""","""710""","""Southern Africa""","""South Africa""","""Western Cape""","""Cape Winelands""","""Drakenstein""","""Wellington""",-33.6398,19.0112,1,"""Cape Argus""","""Subnational""","""On 13 January 2020 (on or arou…",0,"""crowd size=no report""",1579553213
"""SAF12095""","""2020-01-17""",2020,1,"""Demonstrations""","""Riots""","""Violent demonstration""","""Rioters (South Africa)""","""""","""Rioters""","""""","""""","""""","""Rioters only""","""""","""710""","""Southern Africa""","""South Africa""","""Eastern Cape""","""Buffalo City""","""Buffalo City""","""Macleantown""",-32.7886,27.7469,1,"""Daily Dispatch""","""Subnational""","""On 17 January 2020, parents of…",0,"""crowd size=no report""",1579553213
"""SAF12088""","""2020-01-13""",2020,1,"""Demonstrations""","""Protests""","""Peaceful protest""","""Protesters (South Africa)""","""""","""Protesters""","""""","""""","""""","""Protesters only""","""""","""710""","""Southern Africa""","""South Africa""","""Limpopo""","""Capricorn""","""Polokwane""","""Seshego""",-23.8466,29.388,1,"""Daily Sun (South Africa)""","""National""","""On 13 January 2020, community …",0,"""crowd size=no report""",1579553213
"""SAF12092""","""2020-01-15""",2020,1,"""Demonstrations""","""Protests""","""Peaceful protest""","""Protesters (South Africa)""","""""","""Protesters""","""""","""""","""""","""Protesters only""","""""","""710""","""Southern Africa""","""South Africa""","""North West""","""Bojanala""","""Moretele""","""Mmukubyane""",-25.0015,27.9667,2,"""Daily Sun (South Africa)""","""National""","""On 15 January 2020, residents …",0,"""crowd size=no report""",1618515569
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""IDN2259""","""2018-12-10""",2018,1,"""Demonstrations""","""Protests""","""Peaceful protest""","""Protesters (Indonesia)""","""Students (Indonesia)""","""Protesters""","""""","""""","""""","""Protesters only""","""""","""360""","""Southeast Asia""","""Indonesia""","""South Sulawesi""","""Tana Toraja""","""Makale""","""Makale""",-3.1055,119.8546,1,"""Tribunnews""","""National""","""On 10 December 2018, dozens of…",0,"""crowd size=dozens""",1743545562
"""IDN654""","""2018-04-25""",2018,1,"""Political violence; Demonstrat…","""Protests""","""Excessive force against protes…","""Protesters (Indonesia)""","""""","""Protesters""","""Police Forces of Indonesia (20…","""""","""State forces""","""State forces-Protesters""","""Civilian targeting""","""360""","""Southeast Asia""","""Indonesia""","""East Nusa Tenggara""","""West Sumba""","""Lamboya""","""Patiala Bawa""",-9.7554,119.3259,1,"""Amnesty International; Global …","""Other""","""On 25 April 2018, around 100 p…",1,"""crowd size=around 100""",1744744113
"""IDN20917""","""2018-04-10""",2018,1,"""Political violence""","""Violence against civilians""","""Attack""","""Unidentified Armed Group (Indo…","""""","""Political militia""","""Civilia

In [40]:
acled_slice = acled.with_columns(
      pl.col('event_date')
      .str.to_date()
      .dt.strftime("%Y-%m")
      .alias('year_month')).filter(
        (pl.col("iso") == iso)
        & (pl.col("year_month") >= start)
        & (pl.col("year_month") <= end)
    )

NameError: name 'iso' is not defined

In [39]:
filtered

admin1,len
str,u32
"""Veracruz de Ignacio de la Llav…",825
"""Colima""",300
"""Sonora""",351
"""Campeche""",150
"""Puebla""",604
…,…
"""Morelos""",379
"""Durango""",122
"""San Luis Potosi""",111


In [ ]:
from geoacled.utils.fetch import fetch_geojson
from geoacled.
from geoacled.geoacled_types import FeatureCollection
import geopandas as gpd

In [5]:
gdf

,geometry,shapeName,shapeISO,shapeID,shapeGroup,shapeType
0,"MULTIPOLYGON (((-6.59492 54.04459, -6.58608 54...",Ulster,IE-U,36931253B39633918324997,IRL,ADM1
1,"MULTIPOLYGON (((-6.00754 53.49885, -6.00789 53...",Leinster,IE-L,36931253B83705839043930,IRL,ADM1
2,"MULTIPOLYGON (((-7.08588 52.25176, -7.08568 52...",Munster,IE-M,36931253B96342403647060,IRL,ADM1
3,"MULTIPOLYGON (((-7.94658 54.30429, -7.95143 54...",Connacht,IE-C,36931253B96239511377188,IRL,ADM1


In [15]:
lst = [10,12,13,51]
acled.select('iso').unique().to_dicts()

[{'iso': '372'},
 {'iso': '152'},
 {'iso': '191'},
 {'iso': '360'},
 {'iso': '352'},
 {'iso': '705'},
 {'iso': '380'},
 {'iso': '036'},
 {'iso': '032'},
 {'iso': '643'},
 {'iso': '826'},
 {'iso': '840'},
 {'iso': '752'},
 {'iso': '076'},
 {'iso': '250'},
 {'iso': '710'},
 {'iso': '528'},
 {'iso': '724'},
 {'iso': '170'},
 {'iso': '392'},
 {'iso': '428'},
 {'iso': '208'},
 {'iso': '376'},
 {'iso': '682'},
 {'iso': '246'},
 {'iso': '616'},
 {'iso': '356'},
 {'iso': '056'},
 {'iso': '124'},
 {'iso': '233'},
 {'iso': '578'},
 {'iso': '703'},
 {'iso': '040'},
 {'iso': '442'},
 {'iso': '554'},
 {'iso': '348'},
 {'iso': '300'},
 {'iso': '620'},
 {'iso': '203'},
 {'iso': '276'},
 {'iso': '410'},
 {'iso': '440'},
 {'iso': '792'},
 {'iso': '156'},
 {'iso': '484'},
 {'iso': '188'},
 {'iso': '756'}]

In [219]:
nightsky = pl.read_csv('data/ntl_adm2_monthly_2016.csv', null_values='NA')

In [7]:
nightsky

ISO_A3,ADM1CD_c,ADM2CD_c,NAM_0,NAM_1,NAM_2,date,n_gasflaring_locs,ntl_quality_prop_na,ntl_quality_prop_0_good,ntl_quality_prop_1_poor,ntl_quality_prop_2_gapfilled,ntl_sum,ntl_mean,ntl_median,ntl_max,ntl_q05,ntl_q95,ntl_gf_5km_sum,ntl_gf_5km_mean,ntl_gf_5km_median,ntl_gf_5km_max,ntl_gf_5km_q05,ntl_gf_5km_q95,ntl_nogf_5km_sum,ntl_nogf_5km_mean,ntl_nogf_5km_median,ntl_nogf_5km_max,ntl_nogf_5km_q05,ntl_nogf_5km_q95,ntl_gf_10km_sum,ntl_gf_10km_mean,ntl_gf_10km_median,ntl_gf_10km_max,ntl_gf_10km_q05,ntl_gf_10km_q95,ntl_nogf_10km_sum,ntl_nogf_10km_mean,ntl_nogf_10km_median,ntl_nogf_10km_max,ntl_nogf_10km_q05,ntl_nogf_10km_q95
str,str,str,str,str,str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-01-01""",0,0.0,0.874351,0.125649,0.0,5212.629395,6.044495,2.521918,90.729057,0.070298,22.623181,null,null,null,null,null,null,5212.629395,6.044495,2.521918,90.729057,0.070298,22.623181,null,null,null,null,null,null,5212.629395,6.044495,2.521918,90.729057,0.070298,22.623181
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-02-01""",0,0.0,0.906542,0.09242,0.001038,5044.547852,5.84959,2.771983,84.466125,0.096552,20.592231,null,null,null,null,null,null,5044.547852,5.84959,2.771983,84.466125,0.096552,20.592231,null,null,null,null,null,null,5044.547852,5.84959,2.771983,84.466125,0.096552,20.592231
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-03-01""",0,0.0,0.877466,0.122534,0.0,5555.727051,6.442346,3.145681,124.96019,0.181422,21.818279,null,null,null,null,null,null,5555.727051,6.442346,3.145681,124.96019,0.181422,21.818279,null,null,null,null,null,null,5555.727051,6.442346,3.145681,124.96019,0.181422,21.818279
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-04-01""",0,0.0,0.0,1.0,0.0,5187.452148,6.0153,2.89655,69.404099,0.24911,20.452081,null,null,null,null,null,null,5187.452148,6.0153,2.89655,69.404099,0.24911,20.452081,null,null,null,null,null,null,5187.452148,6.0153,2.89655,69.404099,0.24911,20.452081
"""ABW""","""ABW001""","""ABW001001""","""Aruba (Neth.)""","""Administrative unit not availa…","""Administrative unit not availa…","""2016-05-01""",0,0.0,0.0,0.876428,0.123572,5010.268555,5.80984,3.408621,94.178596,0.210237,19.1727,null,null,null,null,null,null,5010.268555,5.80984,3.408621,94.178596,0.210237,19.1727,null,null,null,null,null,null,5010.268555,5.80984,3.408621,94.178596,0.210237,19.1727
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZWE""","""ZWE008""","""ZWE008003""","""Zimbabwe""","""Matabeleland North""","""Hwange""","""2016-12-01""",0,0.000047,0.0,0.683857,0.316095,1830.144897,0.012482,0.251133,105.533417,0.025113,0.477152,null,null,null,null,null,null,1830.144897,0.012482,0.251133,105.533417,0.025113,0.477152,null,null,null,null,null,null,1830.144897,0.012482,0.251133,105.533417,0.025113,0.477152
"""ZWE""","""ZWE009""","""ZWE009005""","""Zimbabwe""","""Matabeleland South""","""Matobo""","""2016-12-01""",0,0.0,0.000108,0.96191,0.037982,153.748627,0.004247,0.252224,28.51372,0.025222,0.479225,null,null,null,null,null,null,153.748627,0.004247,0.252224,28.51372,0.025222,0.479225,null,null,null,null,null,null,153.748627,0.004247,0.252224,28.51372,0.025222,0.479225
"""ZWE""","""ZWE009""","""ZWE009007""","""Zimbabwe""","""Matabeleland South""","""Bulilima""","""2016-12-01""",0,0.0,0.000526,0.987076,0.012397,85.868095,0.002562,0.256001,3.067989,0.0256,0.486401,null,null,null,null,null,null,85.868095,0.002562,0.256001,3.067989,0.0256,0.486401,null,null,null,null,null,null,85.868095,0.002562,0.256001,

In [4]:
nightsky_date = year_month_col(nightsky, 'date')

In [6]:
nightsky_date['ISO_A3', 'ntl_sum', 'year_month'].group_by('ISO_A3', 'year_month').mean()

ISO_A3,year_month,ntl_sum
str,str,f64
"""BRB""","""2016-05""",927.857848
"""BRN""","""2016-09""",1757.215505
"""BTN""","""2016-10""",37.516818
"""BVT""","""2016-07""",1.92469
"""CCK""","""2016-03""",0.0
…,…,…
"""ZAF""","""2016-01""",29690.783609
"""ZAF""","""2016-05""",30393.661642
"""ZAF""","""2016-12""",30225.746505
